# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
print(geoapify_key)

b54d0be010cf40dba0a7ab38200a040b


In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,copainala,17.0833,-93.2000,20.43,99,95,1.09,MX,2023-11-30 03:29:49
1,1,port-aux-francais,-49.3500,70.2167,2.20,97,100,21.56,TF,2023-11-30 03:29:50
2,2,waitangi,-43.9535,-176.5597,12.68,74,21,4.02,NZ,2023-11-30 03:29:50
3,3,yellowknife,62.4560,-114.3525,-17.55,95,40,2.57,CA,2023-11-30 03:26:25
4,4,blackmans bay,-43.0167,147.3167,16.96,61,100,4.63,AU,2023-11-30 03:29:50


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:
%%capture --no-display
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color="City",
    alpha=.5,
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    title = "Humidity of Cities"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Example criteria: Cities with humidity > 50 and latitude between 20 and 40
filtered_cities_df = city_data_df[(city_data_df['Humidity'] > 50) & 
                                  (city_data_df['Lat'] >= 20) & 
                                  (city_data_df['Lat'] <= 40)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
# filtered_cities_df.head()
display(filtered_cities_df)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,ribeira grande,38.5167,-28.7000,14.87,69,84,8.49,PT,2023-11-30 03:29:52
12,12,howrah,22.5892,88.3103,24.00,83,2,2.06,IN,2023-11-30 03:29:52
23,23,san patricio,28.0170,-97.5169,20.91,87,100,5.41,US,2023-11-30 03:29:55
25,25,al bawiti,28.3492,28.8659,13.23,58,78,2.26,EG,2023-11-30 03:29:55
70,70,newman,37.3138,-121.0208,11.83,78,7,2.63,US,2023-11-30 03:27:12
...,...,...,...,...,...,...,...,...,...,...
541,541,la'ie,21.6477,-157.9253,24.66,85,70,3.12,US,2023-11-30 03:32:59
546,546,naha,26.2125,127.6811,26.38,57,75,5.14,JP,2023-11-30 03:29:26
549,549,patterson,37.4716,-121.1297,11.79,80,6,2.10,US,2023-11-30 03:33:01
551,551,ixtapa,20.7000,-105.2000,26.57,83,0,1.54,MX,2023-11-30 03:33:01


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame hotel_df
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
10,ribeira grande,PT,38.5167,-28.7000,69,
12,howrah,IN,22.5892,88.3103,83,
23,san patricio,US,28.0170,-97.5169,87,
25,al bawiti,EG,28.3492,28.8659,58,
70,newman,US,37.3138,-121.0208,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ribeira grande - nearest hotel: Hospedaria JSF
howrah - nearest hotel: Sun India Guest House
san patricio - nearest hotel: No hotel found
al bawiti - nearest hotel: Old Oasis Hotel
newman - nearest hotel: No hotel found
stephenville - nearest hotel: No hotel found
modasa - nearest hotel: No hotel found
ormara - nearest hotel: No hotel found
terrell - nearest hotel: Baymont
hamilton - nearest hotel: North Vista Manor
birjand - nearest hotel: مهمانسرای جهانگردی بیرجند
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
praia da vitoria - nearest hotel: Salles
meiktila - nearest hotel: Honey Hotel
agri - nearest hotel: No hotel found
vicksburg - nearest hotel: Relax Inn Downtown Vicksburg
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
shushtar - nearest hotel: هتل سنتی سرابی شوشتر
ponta delgada - nearest hotel: Hotel Talisman
enoch - nearest hotel: Travelodge by Wyndham Cedar City
lihue - nearest hotel: Kauai Palms
cumra - nearest hotel: No

,City,Country,Lat,Lng,Humidity,Hotel Name
10,ribeira grande,PT,38.5167,-28.7000,69,Hospedaria JSF
12,howrah,IN,22.5892,88.3103,83,Sun India Guest House
23,san patricio,US,28.0170,-97.5169,87,No hotel found
25,al bawiti,EG,28.3492,28.8659,58,Old Oasis Hotel
70,newman,US,37.3138,-121.0208,78,No hotel found
...,...,...,...,...,...,...
541,la'ie,US,21.6477,-157.9253,85,No hotel found
546,naha,JP,26.2125,127.6811,57,ロコアナハ
549,patterson,US,37.4716,-121.1297,80,Hampton Inn & Suites
551,ixtapa,MX,20.7000,-105.2000,83,Ma. Cristina


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display
# Configure the map plot
hotel_df_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color="City",
    alpha=.5,
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    hover_cols= ["Hotel Name", "Country"]
)

# Display the map
hotel_df_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)